In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
result_list = []
for num in range(1, 516):
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL'
    params = {
        'page' : num
    }
    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content)
    tr_list = soup.find_all('tr')[1:]
    for tr in tr_list:
        date = tr.find_all('td')[0].text.strip()
        price = tr.find_all('td')[1].text
        
        new_dict = {
            'date' : date,
            'price' : price
        }
        result_list.append(new_dict)

In [13]:
df = pd.DataFrame(result_list)
df.head()

,date,price
0,2019.07.12,1.74
1,2019.07.11,1.75
2,2019.07.10,1.74
3,2019.07.09,1.69
4,2019.07.08,1.73


In [15]:
import os

In [16]:
BASE_DIR = os.getcwd()
SAVE_DB_DIR = os.path.join(BASE_DIR, 'call')

if not os.path.exists(SAVE_DB_DIR):
    os.makedirs(SAVE_DB_DIR)

#텍스트 파일 만들기
temp_name = f"call.xlsx"
txt_name = os.path.join(SAVE_DB_DIR, temp_name)
excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')
#     df_report.to_excel(excel_writer, sheet_name='report')
df.to_excel(excel_writer)

excel_writer.save()